# Using Tester to test FastKafka

In [ ]:
# | hide

from fastkafka._application.app import FastKafka
from fastkafka._application.tester import Tester
from pydantic import BaseModel, Field
from typing import List, Optional

In order to speed up development and make testing easier, we have implemented the [Tester](../../api/fastkafka/testing/Tester/) class. The [Tester](../../api/fastkafka/testing/Tester/) instance starts InMemory implementation of Kafka broker i.e. there is no need for starting localhost Kafka service for testing FastKafka apps.
The [Tester](../../api/fastkafka/testing/Tester/) will redirect consumes and produces decorated functions to the InMemory Kafka broker so that you can quickly test FasKafka apps without the need of a running Kafka broker and all its dependencies.

**Note**: if you are using Jupyter Notebooks you need to allow the usage of nested event loops when working with asyncio. This can be done by adding the following code:

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Test message

To showcase the functionalities of FastKafka and illustrate the concepts discussed, we can use a simple test message called `TestMsg`. Here's the definition of the `TestMsg` class:

In [ ]:
class TestMsg(BaseModel):
    msg: str = Field(...)

test_msg = TestMsg(msg="signal")

### InMemory Producer and Consumer apps

Let's create two simple `FastKafka` apps. `consumer_app` consumes from the `preprocessed_data` topic and `producer_app` produces messages to the `preprocessed_data` topic.

In [ ]:
from pydantic import BaseModel, Field

consumer_app = FastKafka()


@consumer_app.consumes()
async def on_preprocessed_data(msg: TestMsg):
    print(msg)


producer_app = FastKafka()


@producer_app.produces()
async def to_preprocessed_data(msg: TestMsg) -> TestMsg:
    return msg

By using `Tester([consumer_app, producer_app])`, this two apps will use InMemory Kakfa broker.

**Note**: it is necessary to define parameter and return types in the producer and consumer methods

In [ ]:
test_msg = TestMsg(msg="signal")
async with Tester([consumer_app, producer_app]) as tester:
    # producer_app produces message to the preprocessed_data topic
    await producer_app.to_preprocessed_data(test_msg)
    # assert that consumer_app consumed from the preprocessed_data topic and it was called with test_msg argument
    await consumer_app.awaited_mocks.on_preprocessed_data.assert_called_with(
        test_msg, timeout=5
    )
print("ok")

### Final script

**Note**: If you are running these examples in `.py` files, [Tester](../../api/fastkafka/testing/Tester/) needs to be in the `async` function (you can't run async code in the sync function), for Jupyther Notebooks this isn't necessary. Testing `.py` script example:

In [ ]:
import asyncio
from fastkafka._application.app import FastKafka
from fastkafka._application.tester import Tester
from pydantic import BaseModel, Field


class TestMsg(BaseModel):
    msg: str = Field(...)

consumer_app = FastKafka()

@consumer_app.consumes()
async def on_preprocessed_data(msg: TestMsg):
    print(msg)


producer_app = FastKafka()


@producer_app.produces()
async def to_preprocessed_data(msg: TestMsg) -> TestMsg:
    return msg


async def async_tests():
    test_msg = TestMsg(msg="signal")
    async with Tester([consumer_app, producer_app]) as tester:
        # producer_app produces message to the preprocessed_data topic
        await producer_app.to_preprocessed_data(test_msg)
        # assert that consumer_app consumed from the preprocessed_data topic and it was called with test_msg argument
        await consumer_app.awaited_mocks.on_preprocessed_data.assert_called_with(
            test_msg, timeout=5
    )
    print("ok")

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(async_tests())

### Consumer app + Tester (producer) app

It is often necessary to implement only the Kakfa consumer, which then subscribes to an already existing topic. The [Tester](../../api/fastkafka/testing/Tester/) enables simple mirroring, i.e. if we have implemented consumer, the [Tester](../../api/fastkafka/testing/Tester/) will mock the producer methods (and the other way - if the producer is implemented, [Tester](../../api/fastkafka/testing/Tester/) will mock consumer methods).
In this examle we have `consumer_app` which consumes from the `preprocessed_data` topic.

In [ ]:
consumer_app = FastKafka()


@consumer_app.consumes()
async def on_preprocessed_data(msg: TestMsg):
    print(msg)

Here `consumer_app` has implemented `on_preprocessed_data` and `tester` will have consumer mirrored methods i.e. `to_preprocessed_data` (the same way `producer_app` had `to_preprocessed_data` in the previous example)

In [ ]:
async with Tester(consumer_app) as tester:
    # Send the message to the topic preprocessed_data
    await tester.to_preprocessed_data(test_msg)
    # await tester.mirrors[consumer_app.on_preprocessed_data](test_msg) # the same as previous line

    # Tester mirrors consumer_app consumes() and produces() methods (in this example there is only on_preprocessed_data)
    assert (
        tester.to_preprocessed_data == tester.mirrors[consumer_app.on_preprocessed_data]
    )

    # Assert app consumed the message from the preprocessed_data topic
    await consumer_app.awaited_mocks.on_preprocessed_data.assert_called_with(
        test_msg, timeout=5
    )
print("ok")

### Final script

In [ ]:
import asyncio
from fastkafka._application.app import FastKafka
from fastkafka._application.tester import Tester
from pydantic import BaseModel, Field


class TestMsg(BaseModel):
    msg: str = Field(...)

consumer_app = FastKafka()

@consumer_app.consumes()
async def on_preprocessed_data(msg: TestMsg):
    print(msg)

async def async_tests():
    test_msg = TestMsg(msg="signal")
    async with Tester(consumer_app) as tester:
        # Send the message to the topic preprocessed_data
        await tester.to_preprocessed_data(test_msg)
        # await tester.mirrors[consumer_app.on_preprocessed_data](test_msg) # the same as previous line

        # Tester mirrors consumer_app consumes() and produces() methods (in this example there is only on_preprocessed_data)
        assert (
            tester.to_preprocessed_data == tester.mirrors[consumer_app.on_preprocessed_data]
        )

        # Assert app consumed the message from the preprocessed_data topic
        await consumer_app.awaited_mocks.on_preprocessed_data.assert_called_with(
            test_msg, timeout=5
        )
    print("ok")
    

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(async_tests())

### Producer app + Tester (consumer) app

In this example [Tester](../../api/fastkafka/testing/Tester/) will mirror producer method the same way it mirrored the consumer method in the previous example

In [ ]:
producer_app = FastKafka()


@producer_app.produces()
async def to_preprocessed_data(msg: TestMsg) -> TestMsg:
    return msg

In [ ]:
async with Tester(producer_app) as tester:
    # Send the message to the topic preprocessed_data
    await producer_app.to_preprocessed_data(test_msg)

    # Assert app consumed the message from the preprocessed_data topic
    await tester.awaited_mocks.on_preprocessed_data.assert_called_with(
        test_msg, timeout=5
    )
print("ok")

### Final script

In [ ]:
import asyncio
from fastkafka._application.app import FastKafka
from fastkafka._application.tester import Tester
from pydantic import BaseModel, Field


class TestMsg(BaseModel):
    msg: str = Field(...)

producer_app = FastKafka()

@producer_app.produces()
async def to_preprocessed_data(msg: TestMsg) -> TestMsg:
    return msg

async def async_tests():
    test_msg = TestMsg(msg="signal")
    async with Tester(producer_app) as tester:
        # Send the message to the topic preprocessed_data
        await producer_app.to_preprocessed_data(test_msg)

        # Assert app consumed the message from the preprocessed_data topic
        await tester.awaited_mocks.on_preprocessed_data.assert_called_with(
            test_msg, timeout=5
        )
    print("ok")
    

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(async_tests())

### App (producer & consumer)  + Tester (producer & consumer) app #1

In this example [Tester](../../api/fastkafka/testing/Tester/) will mirror producer and consumer methods.

In [ ]:
app = FastKafka()


@app.consumes()
async def on_preprocessed_data(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> List[TestMsg]:
    print(f"Sending prediction: {prediction}")
    return [prediction]

The `app` has `on_preprocessed_data` and `to_predictions` method defined and `tester` will have their mirrored methods: `to_preprocessed_data` and `on_predictions`.

In [ ]:
async with Tester(app) as tester:
    # inside on_preprocessed_data, to_predictions method is called i.e. app will produce message to predictions topic
    await app.on_preprocessed_data(test_msg)
    # Now we can check if tester consumed from the predictions topic
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)

    # produce message to preprocessed_data topic
    await tester.to_preprocessed_data(test_msg)
    # app will consume from the preprocessed_data topic
    await app.awaited_mocks.on_preprocessed_data.assert_called_with(test_msg, timeout=5)
    # while consuming from the preprocessed_data topic (in the line above), app will produce a message to predictions topic
    # and tester will cosume this message
    tester.mocks.on_predictions.assert_called()
print("ok")

### Final script

In [ ]:
import asyncio
from fastkafka._application.app import FastKafka
from fastkafka._application.tester import Tester
from pydantic import BaseModel, Field
from typing import List

class TestMsg(BaseModel):
    msg: str = Field(...)

app = FastKafka()


@app.consumes()
async def on_preprocessed_data(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> List[TestMsg]:
    print(f"Sending prediction: {prediction}")
    return [prediction]


async def async_tests():
    test_msg = TestMsg(msg="signal")
    async with Tester(app) as tester:
        # inside on_preprocessed_data, to_predictions method is called i.e. app will produce message to predictions topic
        await app.on_preprocessed_data(test_msg)
        # Now we can check if tester consumed from the predictions topic
        await tester.awaited_mocks.on_predictions.assert_called(timeout=5)

        # produce message to preprocessed_data topic
        await tester.to_preprocessed_data(test_msg)
        # app will consume from the preprocessed_data topic
        await app.awaited_mocks.on_preprocessed_data.assert_called_with(test_msg, timeout=5)
        # while consuming from the preprocessed_data topic (in the line above), app will produce a message to predictions topic
        # and tester will cosume this message
        tester.mocks.on_predictions.assert_called()
    print("ok")

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(async_tests())

### App (producer & consumer)  + Tester (producer & consumer) app #2

In this example, our `app` has one consumes and two produces methods.
Every time a company hires new `Employee`, the new employee needs to get email address `on_new_employee`. Also, we want to send a welcome mail to the new employee `to_welcome_message`


**Note**: [Tester](../../api/fastkafka/testing/Tester/) uses `unittest.mock` module for the creation of mirrored methods. In this example, we defined `on_new_employee(msg: Employee)`, `tester` will create `to_new_employee(msg: Employee) -> Employee` and both methods will have a POINTER TO THE SAME OBJECT `msg: Employee`. This problem can be solved easily by creating a copy of `msg` object at the beggining of `on_new_employee` method -> `msg_copy = msg.copy()`

In [ ]:
class Employee(BaseModel):
    name: str
    surname: str
    email: Optional[str]


class EmaiMessage(BaseModel):
    sender: str = "info@gmail.com"
    receiver: str
    subject: str
    message: str


app = FastKafka()


@app.consumes()
async def on_new_employee(msg: Employee):
    msg_copy = msg.copy()
    employee = await to_employee_email(msg_copy)
    await to_welcome_message(employee)


@app.produces()
async def to_employee_email(employee: Employee) -> Employee:
    # generate new email
    employee.email = employee.name + "." + employee.surname + "@gmail.com"
    return employee


@app.produces()
async def to_welcome_message(employee: Employee) -> EmaiMessage:
    message = f"Dear {employee.name},\nWelcome to the company"
    return EmaiMessage(receiver=employee.email, subject="Welcome", message=message)

In [ ]:
async with Tester(app) as tester:
    # while consuming from the new_employee topic, app writes to the employee_email and welcome_message topic
    await app.on_new_employee(Employee(name="John", surname="Jones"))

    # Now we can check if the methods were called
    await tester.awaited_mocks.on_employee_email.assert_called(timeout=5)
    await tester.awaited_mocks.on_welcome_message.assert_called(timeout=5)

print("ok")

In [ ]:
async with Tester(app) as tester:
    # produce the message to new_employee topic
    await tester.to_new_employee(Employee(name="Mickey", surname="Mouse"))
    # Assert app consumed the message
    await app.awaited_mocks.on_new_employee.assert_called_with(
        Employee(name="Mickey", surname="Mouse"), timeout=5
    )

    # If the the previous assert is true (on_new_employee was called),
    # to_employee_email and to_welcome_message were called inside on_new_employee method

    # Now we can check if this two messages were consumed
    await tester.awaited_mocks.on_employee_email.assert_called(timeout=5)
    await tester.awaited_mocks.on_welcome_message.assert_called(timeout=5)

print("ok")

### Final script

In [ ]:
import asyncio
from fastkafka._application.app import FastKafka
from fastkafka._application.tester import Tester
from pydantic import BaseModel, Field
from typing import Optional


class Employee(BaseModel):
    name: str
    surname: str
    email: Optional[str]


class EmaiMessage(BaseModel):
    sender: str = "info@gmail.com"
    receiver: str
    subject: str
    message: str


app = FastKafka()


@app.consumes()
async def on_new_employee(msg: Employee):
    msg_copy = msg.copy()
    employee = await to_employee_email(msg_copy)
    await to_welcome_message(employee)


@app.produces()
async def to_employee_email(employee: Employee) -> Employee:
    # generate new email
    employee.email = employee.name + "." + employee.surname + "@gmail.com"
    return employee


@app.produces()
async def to_welcome_message(employee: Employee) -> EmaiMessage:
    message = f"Dear {employee.name},\nWelcome to the company"
    return EmaiMessage(receiver=employee.email, subject="Welcome", message=message)

async def async_tests():
    async with Tester(app) as tester:
        # while consuming from the new_employee topic, app writes to the employee_email and welcome_message topic
        await app.on_new_employee(Employee(name="John", surname="Jones"))

        # Now we can check if the methods were called
        await tester.awaited_mocks.on_employee_email.assert_called(timeout=5)
        await tester.awaited_mocks.on_welcome_message.assert_called(timeout=5)

    print("ok 1")

    async with Tester(app) as tester:
        # produce the message to new_employee topic
        await tester.to_new_employee(Employee(name="Mickey", surname="Mouse"))
        # Assert app consumed the message
        await app.awaited_mocks.on_new_employee.assert_called_with(
            Employee(name="Mickey", surname="Mouse"), timeout=5
        )

        # If the the previous assert is true (on_new_employee was called),
        # to_employee_email and to_welcome_message were called inside on_new_employee method

        # Now we can check if this two messages were consumed
        await tester.awaited_mocks.on_employee_email.assert_called(timeout=5)
        await tester.awaited_mocks.on_welcome_message.assert_called(timeout=5)

    print("ok 2")

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(async_tests())